In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import sys
sys.path.append("../lib/")
from pathlib import Path

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

2023-05-15 10:21:59.336605: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 10:21:59.440248: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 10:21:59.441363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 10:22:00.629671: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.0-dev20230411


In [2]:
dataset = "L_540_2022_C_E_R_B"
results_path = Path("../results/") / dataset
df = pd.read_csv(results_path / (dataset + "_preprocessed.csv"))


X = pd.get_dummies(df.drop(['label', 'file'], axis=1))

# Map the class names to integer values
class_map = {'Both': 0, 'Control': 1, 'Etoposide': 2, 'Resveratrol': 3}
y = df['label'] = df['label'].replace(class_map)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [5]:
class_names = ['Both', 'Control', 'Etoposide', 'Resveratrol']

# Build the model

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2601,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model

In [11]:
model.fit(X_train, y_train, epochs=100, use_multiprocessing = True, verbose = False)

In [12]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

8/8 - 0s - loss: 0.9775 - accuracy: 0.6625 - 207ms/epoch - 26ms/step

Test accuracy: 0.6625000238418579


# Build the model

In [38]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)
predictions[0]

8/8 [==============================] - 0s 2ms/step


array([4.2935785e-06, 9.5329696e-01, 2.3143338e-02, 2.3555385e-02],
      dtype=float32)

In [39]:
np.argmax(predictions[0])

1

# Use the trained model

In [60]:
# Grab an image from the test dataset.
test = X_test[:1]

# Tune model

In [ ]:
model.save('tfmodel')
del model 
model = load_model('tfmodel')